## Accessing PAIRS Image Data through API 


Before importing api_wrapper, had to install some dependencies: 
- pip install geopandas, geojson, coloredlogs
- conda install gdal

In [1]:
import sys
import os
# save present working directory so we can switch back to it 
pwd = os.getcwd()
# change directory to get access to api_wrapper
os.chdir('../scalable-landcover-classification/Querying')
import api_wrapper as paw
# switch back to original directory
os.chdir(pwd)
import pandas as pd
import numpy as np
import geojson
import urllib
from datetime import datetime
from pytz import UTC
import time
import random
import logging
import requests
from requests.auth import HTTPBasicAuth
import os, zipfile, io
from osgeo import gdal
import PIL

Config

In [7]:
PAIRS_USER = "ams792@cornell.edu"
PAIRS_PASS = "myibmpairspassword"
server = "https://pairs.res.ibm.com"
auth = (PAIRS_USER, PAIRS_PASS)
path = './downloads' #Download path

In [10]:
lat1 = 41.0100756423
lat2 = 41.0338409682
lon1 = -73.7792749922
lon2 = -73.7582464736

In [26]:
response = requests.post(
    json = {
    "layers" : [
        {
            "type": "raster",
            "id": 35445
        },
        {
            "type": "raster",
            "id": 36431
        },
        {
            "type": "raster",
            "id": 36432
        },
        {
            "type": "raster",
            "id": 49104
        }
    ],
    "spatial": {
        "type": "square",
        "coordinates": ["41.0100746", "-73.779274", "41.0338402", "-73.7582474"]
    },
    "temporal": {
        "intervals": [{"start": "2014-12-31T00:00:00Z","end": "2015-01-01T00:00:00Z"}]
    }
},
    url=f'{server}/v2/query',
    auth=auth,
)

In [27]:
res = response.json()
id = res['id']
print("ID: ",id)

ID:  1539792000_15574391


In [28]:
response

<Response [200]>

Check status of query, make sure it's finished before downloading

In [31]:
response = requests.get(
    url=f'{server}/v2/queryjobs/{id}',
    auth=auth,
)
response.json()

{'exPercent': 0.0,
 'flag': False,
 'folder': '1539807574390',
 'hadoopId': '1535736177095_92515',
 'id': '1539792000_15574391',
 'neLat': 41.0338402,
 'neLon': -73.7582474,
 'nickname': None,
 'pdStatus': None,
 'pql': None,
 'ready': False,
 'rtStatus': 'Running',
 'start': 1539807574391,
 'status': 'Running',
 'statusCode': 10,
 'swLat': 41.0100746,
 'swLon': -73.779274}

Download and extract to files

In [140]:
download = requests.get(
    f'{server}/download/{id}', auth=auth, stream=True,
)

z = zipfile.ZipFile(io.BytesIO(download.content))

## Extract to ./downloads
z.extractall(path)

Convert to np array

In [141]:
## Takes file name and converts to np array
def tiff2array(fn):
    ## Special gdal dataset
    dataset = gdal.Open(fn)
    array = np.array(dataset.GetRasterBand(1).ReadAsArray())
    return array

## Will go through filenames and put the image arrays in here
images_dict = {}

## Loop through files in downloads directory (if multiple)
for i, filename in enumerate(os.listdir(path)):
    if filename.endswith(".tiff"): 
        path_to_file = path + '/' + filename
        images_dict[i] = tiff2array(path_to_file)

## Note: keys start at 1 for image_dict



In [143]:
res = requests.get(f'{server}/v2/datalayers/',auth=auth)

In [100]:
res.json()

[{'dataType': 'fl',
  'dataset': {'category': {'id': 2, 'name': 'Weather'},
   'crs': 'EPSG:4326',
   'id': 87,
   'name': 'Snow Coverage'},
  'id': '48524',
  'name': 'Snow depth',
  'unit': 'mm'},
 {'dataType': 'fl',
  'dataset': {'category': {'id': 4, 'name': 'Analytics'},
   'crs': '',
   'id': 93,
   'name': 'US Soil Properties'},
  'id': '48695',
  'name': 'Silt (50 to 100 cm depth)',
  'unit': None},
 {'dataType': 'fl',
  'dataset': {'category': {'id': 4, 'name': 'Analytics'},
   'crs': '',
   'id': 93,
   'name': 'US Soil Properties'},
  'id': '48696',
  'name': 'Sum (50 to 100 cm depth)',
  'unit': None},
 {'dataType': 'bt',
  'dataset': {'category': {'id': 4, 'name': 'Analytics'},
   'crs': None,
   'id': 33,
   'name': 'Global Land Cover'},
  'id': '49276',
  'name': 'Global Land Cover Climatology',
  'unit': None},
 {'dataType': 'fl',
  'dataset': {'category': {'id': 4, 'name': 'Analytics'},
   'crs': '',
   'id': 93,
   'name': 'US Soil Properties'},
  'id': '48693',
  'na